In [1]:
import os
import logging
import numpy as np
import bpy
from scipy.spatial.transform import Rotation as R
import random
from kubric import randomness

import tensorflow as tf
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import json
import urllib3
import shutil
import tarfile
import sys
import os
os.environ["KUBRIC_USE_GPU"] = "1"

http = urllib3.PoolManager()

import kubric as kb
from kubric.renderer import Blender

In [2]:
devnull = os.open('/dev/null', os.O_WRONLY)
os.dup2(devnull, 1)
os.close(devnull)

In [3]:
shapenet_gs = 'gs://kubric-unlisted/assets/ShapeNetCore.v2/'
manifest_gs = 'gs://kubric-unlisted/assets/ShapeNetCore.v2.json'
manifest_path = Path('ShapeNetCore.v2.json')
shapenet_path = Path('shapenet')
renders_path = Path('new_renders')

In [4]:
def fetch_gs(src_gs, dest_path):
    if not dest_path.exists():
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        tf.io.gfile.copy(src_gs, dest_path)

def fetch_url(src_url, dest_path):
    if not dest_path.exists():
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        with http.request('GET', src_url, preload_content=False) as r, open(dest_path, 'wb') as dest_file:
            shutil.copyfileobj(r, dest_file)

In [5]:
class ShapeNet():
    def __init__(self):
        fetch_gs(manifest_gs, manifest_path)
        with open(manifest_path, 'rb') as manifest_file:
            manifest = json.loads(manifest_file.read())
        self.assets = manifest['assets']
    
    def fetch_asset_tar(self, asset_id):
        path = shapenet_path / (asset_id + ".tar.gz")
        fetch_gs(shapenet_gs + (asset_id + ".tar.gz"), path)
        return path
    
    def fetch_asset(self, asset_id):
        path = shapenet_path / asset_id
        if not path.is_dir():
            tar_path = self.fetch_asset_tar(asset_id)
            with tarfile.open(tar_path) as tar:
                tar.extractall(path)
        return path
    
    def get_category(self, category):
        return [asset_id for asset_id, asset in self.assets.items() if asset['metadata']['category'] == category]
    
    def fetch_category_tar(self, category):
        asset_ids = self.get_category(category)
        for i in tqdm(range(len(asset_ids)), desc='Fetching tar...'):
            asset_id = asset_ids[i]
            self.fetch_asset_tar(asset_id)
    
    def fetch_category(self, category):
        asset_ids = self.get_category(category)
        for i in tqdm(range(len(asset_ids)), desc='Fetching...'):
            asset_id = asset_ids[i]
            self.fetch_asset(asset_id)
    
    def create(self, asset_id):
        asset_entry = self.assets[asset_id]
        asset_type = kb.AssetSource._resolve_asset_type(asset_entry["asset_type"])
        asset_dir = self.fetch_asset(asset_id)

        asset_kwargs = asset_entry.get("kwargs", {})
        asset_kwargs = kb.AssetSource._adjust_paths(asset_kwargs, asset_dir)
        if asset_type == kb.core.FileBasedObject:
            asset_kwargs["asset_id"] = asset_id
        asset = asset_type(**asset_kwargs)
        asset.metadata.update(asset_entry.get("metadata", {}))

        return asset

In [6]:
def setup(camera_angles, resolution):
    scene = kb.Scene(resolution=resolution)
    renderer = Blender(scene=scene, use_denoising=True,
                   samples_per_pixel=64, background_transparency=True)

    wall_material = kb.FlatMaterial(color=kb.get_color('gray'),
                                    indirect_visibility=True, holdout=True)
    walls = [
        kb.Cube(name='floor', scale=(100, 100, 1), position=(0, 0, -10), material=wall_material),
        #kb.Cube(name='nwall', scale=(0.1, 2, 1), position=(-2, 0, 0.9), material=wall_material),
        #kb.Cube(name='ewall', scale=(2, 0.1, 1), position=(0, 2, 0.9), material=wall_material),
        #kb.Cube(name='swall', scale=(0.1, 2, 1), position=(2, 0, 0.9), material=wall_material),
        #kb.Cube(name='wwall', scale=(2, 0.1, 1), position=(0, -2, 0.9), material=wall_material),
        #kb.Cube(name='ceiling', scale=(2, 2, 1), position=(0, 0, 1.9), material=wall_material)
    ]
    for wall in walls:
        scene += wall
        wall.linked_objects[renderer].cycles.is_shadow_catcher = True

    #light_pose = R.from_euler('xz', (50, 70), degrees=True).apply([0, 2, 0])
    #scene += kb.PointLight(name="point_light", position=light_pose, intensity=15)
    
    scene += kb.assets.utils.get_clevr_lights(rng=np.random.RandomState(0))
    #scene.ambient_illumination = kb.Color(0.05, 0.05, 0.05)

    scene.camera = kb.PerspectiveCamera()
    centroid = (0, 0, 0.2)
    camera_poses = [centroid + R.from_euler('xz', (15, a), degrees=True).apply([0, 1.5, 0]) for a in camera_angles]
    scene.frame_end = len(camera_poses)
    for i in range(scene.frame_start, scene.frame_end + 1):
        scene.camera.position = camera_poses[i-1]
        scene.camera.look_at(centroid)
        scene.camera.keyframe_insert("position", i)
        scene.camera.keyframe_insert("quaternion", i)
    
    return scene, renderer

In [7]:
def center_obj(obj):
    obj.quaternion = kb.Quaternion(axis=[1, 0, 0], degrees=90)
    obj.position = (0, 0, obj.position[2] - obj.aabbox[0][2])

In [8]:
def save_frame(data_stack, asset_id, new_metadatas):
    asset_render_path = renders_path / asset_id
    asset_render_path.mkdir(parents=True, exist_ok=True)
    asset_meta_path = asset_render_path / 'metadata.json'
    if asset_meta_path.is_file():
        with open(asset_meta_path, 'r') as f:
            metadatas = json.load(f)
            metadata_list = {int(k): v for k, v in metadata_list.items()}
    else:
        metadatas = {}
    offset = 1 + max(metadatas, default=-1)
    
    kb.file_io.write_rgba_batch(data_stack["rgba"], str(asset_render_path), file_template='rgba_{:05d}.png')
    kb.file_io.write_depth_batch(data_stack["depth"], str(asset_render_path), file_template='depth_{:05d}.png')
    kb.file_io.write_segmentation_batch(data_stack["segmentation"], str(asset_render_path), file_template='segmentation_{:05d}.png')
    
    for i, metadata in enumerate(new_metadatas):
        metadata['id'] = i + offset
        metadata['rgba_filename'] = f"{metadata['name']}_rgba_{metadata['id']:05d}.png"
        metadata['depth_filename'] = f"{metadata['name']}_depth_{metadata['id']:05d}.png"
        metadata['segmentation_filename'] = f"{metadata['name']}_segmentation_{metadata['id']:05d}.png"
        os.rename(asset_render_path / 'rgba_{:05d}.png'.format(i), str(asset_render_path / metadata['rgba_flename']))
        os.rename(asset_render_path / 'depth_{:05d}.png'.format(i), str(asset_render_path / metadata['depth_filename']))
        os.rename(asset_render_path / 'segmentation_{:05d}.png'.format(i), str(asset_render_path / metadata['segmentation_filename']))
        metadatas[metadata['id']] = metadata

    with open(asset_meta_path, 'w') as f:
        json.dump(metadatas, f, indent=4)

In [9]:
def render_batch(resolution, asset_ids, camera_angles=[], num_random=0, angle_range=(0, 360)):
    scene, renderer = setup(camera_angles + [0]*num_random, resolution)
    centroid = (0, 0, 0.2)
    
    for i in tqdm(range(len(asset_ids)), desc='Rendering...'):
        asset_id = asset_ids[i]
        obj = shapenet.create(asset_ids[i])
        center_obj(obj)
        scene.add(obj)
        
        random_angles = (angle_range[0] + np.random.rand(num_random) * (angle_range[1] - angle_range[0])).tolist()
        for j, angle in enumerate(random_angles):
            scene.camera.position = centroid + R.from_euler('xz', (15, angle), degrees=True).apply([0, 1.5, 0])
            scene.camera.look_at(centroid)
            scene.camera.keyframe_insert("position", j+len(camera_angles)+1)
            scene.camera.keyframe_insert("quaternion", j+len(camera_angles)+1)
        
        metadatas = [{ 'angle': angle, 'resolution': resolution } for angle in camera_angles + random_angles]
        for i, angle in enumerate(camera_angles):
            metadatas[i]['name'] = f"angle_{angle}"
        for i in range(len(camera_angles), len(camera_angles) + num_random):
            metadatas[i]['name'] = f"random_{angle_range}"
        data_stack = renderer.render()
    
        kb.compute_visibility(data_stack["segmentation"], scene.assets)
        data_stack["segmentation"] = kb.adjust_segmentation_idxs(
            data_stack["segmentation"],
            scene.assets,
            [obj]).astype(np.uint8)
        save_frame(data_stack, asset_id, metadatas)
        scene.remove(obj)

In [10]:
shapenet = ShapeNet()
cars = shapenet.get_category('car')
failed_ids = [695]
cars = [car for i, car in enumerate(cars) if i not in failed_ids]

In [12]:
cars[695]

'02958343/3973846825fc4d857cb2a55fa21392b7'

In [20]:
camera_angles = [30, 60, 150]
num_random = 5
angle_range = (20, 70)
resolution = (128, 128)
batch_size = 100
batch_indices = list(range(695, len(cars), batch_size))
for start in batch_indices:
    print(f'Rendering batch {start} to {min(start+batch_size, len(cars))}')
    render_batch(resolution, cars[start:start+batch_size], [], num_random, angle_range)

Rendering batch 695 to 795


Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:17<00:00,  4.38s/it]

Rendering batch 795 to 895



Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:19<00:00,  4.39s/it]

Rendering batch 895 to 995


...


Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:30<00:00,  4.51s/it]

Rendering batch 3295 to 3395



Rendering...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:31<00:00,  4.51s/it]

Rendering batch 3395 to 3485



Rendering...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [06:38<00:00,  4.43s/it]


In [18]:
def modify_metadata():
    for asset_id in cars[695:]:
        asset_render_path = renders_path / asset_id
        try:
            with open(asset_render_path / 'metadata_list.json', 'r') as f:
                metadata_list = json.load(f)
        except:
            continue

        for i, metadata in enumerate(metadata_list):
            pass
            
        with open(asset_render_path / 'metadata_list.json', 'w') as f:
            json.dump(metadata_list, f, indent=4)